In [ ]:
!pip install tensorflow

In [1]:
'''步骤一，导入本项目所需要的模块和包，从 Keras 中导入 Cifar-10 数据集，将训练集的特征值和标签值分别存储在
x_train 和 y_train 中，将测试集的特征值和标签值分别存储在 x_test 和 y_test 中。将特征值 x_train 和 x_test 
的数据类型转换为 tf.float32，并进行标准化处理，使其取值范围为 0～1；将标签值 y_train 和 y_test的数据类型转换为 tf.int32。'''

# 导入所需要的模块与包
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os

# 导入Cifar-10数据集，将训练集和测试集存储在相应变量中
cifar10 = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# 转换特征值的数据类型并进行标准化处理
x_train, x_test = tf.cast(x_train, dtype=tf.float32) / 255.0, tf.cast(x_test, dtype=tf.float32) / 255.0

# 转换标签值的数据类型
y_train, y_test = tf.cast(y_train, dtype=tf.int32), tf.cast(y_test, dtype=tf.int32)

# 显示数据集的特征值和标签值的 shape 属性值
print("x_train.shape =", x_train.shape)
print("y_train.shape =", y_train.shape)
print("x_test.shape =", x_test.shape)
print("y_test.shape =", y_test.shape)


ModuleNotFoundError: No module named 'google'

In [ ]:
'''步骤二，构建 CNN 模型，图像识别分类网络模型采用卷积神经网络，包括两
个卷积层、两个池化层、4 个 Dropout 层、一个拉伸层、两个全连接层和一个输
出层'''
# 构建 CNN 模型
model = tf.keras.models.Sequential([
    
    # 使用 Input 层明确指定输入形状
    tf.keras.Input(shape=x_train.shape[1:]),  
    
    # 创建卷积层
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation=tf.nn.relu),
    
    # 创建最大池化层
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='same'),  
    
    # 创建 Dropout 层
    tf.keras.layers.Dropout(0.2),  
    
    # 创建第二个卷积层
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), padding='same', activation=tf.nn.relu),
    
    # 创建最大池化层
    tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='same'),  
    
    # 创建 Dropout 层
    tf.keras.layers.Dropout(0.2),  
    
    # 创建拉伸层（Flatten）
    tf.keras.layers.Flatten(),  
    
    # 创建全连接层
    tf.keras.layers.Dense(512, activation='relu'),  
    
    # 创建 Dropout 层
    tf.keras.layers.Dropout(0.2),  
    
    # 创建另一个全连接层
    tf.keras.layers.Dense(256, activation='relu'),  
    
    # 创建 Dropout 层
    tf.keras.layers.Dropout(0.5),  
    
    # 创建全连接层作为输出层
    tf.keras.layers.Dense(10, activation='softmax')  
])

# 打印模型结构
model.summary()

In [ ]:
'''步骤三，编译、训练和评估 CNN 模型'''
# 编译网络模型
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
              metrics=['sparse_categorical_accuracy'])

# 训练网络模型
history = model.fit(x_train, y_train, 
                    batch_size=128, 
                    epochs=10, 
                    validation_split=0.2)

# 评估网络模型
model.evaluate(x_test, y_test, 
               batch_size=64, 
               verbose=2)

In [ ]:
'''步骤四，可视化训练的结果'''
print(history.history)

# 确保 epochs 数组的定义与 loss 和 acc 的数据一致
epochs = range(1, len(history.history['loss']) + 1)

# 创建画布并设置画面大小
plt.figure(figsize=(10, 3))

# 在子图1中绘制损失函数的折线图
plt.subplot(121)
plt.plot(epochs, history.history['loss'], color='b', label='train')  # 蓝色线表示训练集损失
plt.plot(epochs, history.history['val_loss'], color='r', label='validate')  # 红色线表示验证集损失
plt.xlabel('Epochs')  # 添加横轴标签
plt.ylabel('Loss')  # 添加纵轴标签
plt.legend()  # 显示图例

# 在子图2中绘制准确率的折线图
plt.subplot(122)
plt.plot(epochs, history.history['sparse_categorical_accuracy'], color='b', label='train')  # 蓝色线表示训练集准确率
plt.plot(epochs, history.history['val_sparse_categorical_accuracy'], color='r', label='validate')  # 红色线表示验证集准确率
plt.xlabel('Epochs')  # 添加横轴标签
plt.ylabel('Accuracy')  # 添加纵轴标签
plt.legend()  # 显示图例

# 显示图像
plt.show()



In [ ]:

'''步骤五，应用卷积神经网络模型'''
plt.figure()  # 创建画布

for i in range(10):  # 循环显示10张图片
    n = np.random.randint(1, 10000)  # 生成随机整数 n
    plt.subplot(2, 5, i + 1)  # 创建子图 (2行5列)
    plt.axis("off")  # 不显示坐标轴
    plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置字体为 SimHei，支持中文显示

    # 显示测试集中随机图片，使用灰度颜色映射
    plt.imshow(x_test[n], cmap='gray')

    # 将图片 reshape 为模型输入的形状 (1, 32, 32, 3)
    demo = tf.reshape(x_test[n], (1, 32, 32, 3))

    # 使用模型预测该图片的类别
    y_pred = np.argmax(model.predict(demo))

    # 设置子图标题，显示标签值与预测值
    title = "标签值: " + str((y_test.numpy())[n, 0]) + "\n预测值: " + str(y_pred)
    plt.title(title)

# 展示图像
plt.show()